# CoBlend
**a notebook brought to you by [f1shy-dev](https://github.com/f1shy-dev)**

Blender rendering in Google Colab, now with a fancy, easy-to-use GUI!

To get started, switch to a GPU runtime and just run the massive codeblock below.

**Credits**
* https://github.com/ynshung/blender-colab for the inspiration and for large amounts of the actual blender bits (e.g setgpu.py)

In [9]:
import os
import subprocess
import sys
from google.colab import files, drive
from datetime import datetime
import shutil
print("CoBlend v1.0.0a by f1shy-dev\n")

#-- install/check necessary packages
packages = ["ipyfilechooser", ["rich", 'rich[jupyter]'], 'ipywidgets', 'wget']
for package in packages:
    rp = package[0] if isinstance(package, list) else package
    imp = package[1] if isinstance(package, list) else package
    try:
        __import__(rp)
        print(f"✅ {rp} package installed")
    except ImportError:
        print(f"⚙️ installing package {rp}...")
        subprocess.check_call(['pip', 'install', imp])

from rich import print as rprint

#-- check gpu
gpu = "Unknown"
gpu_optix = True
try:
    gpu = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name', '--format=csv,noheader']).decode().strip()
    if gpu[0] == "Tesla K80":
      gpu_optix = False
      print(f"⚠️ GPU detected - {gpu} (Note: OptiX not supported)")
    else:
      print(f"✅ GPU detcted - {gpu}")
except (subprocess.CalledProcessError, FileNotFoundError):
    rprint("[red3]The nvidia-smi command wasn't found, probably meaning you aren't on a GPU runtime.")
    rprint("[red3]Please switch to a GPU runtime to use this script properly.")
    # TODO: fix CPU rendering...
    # rprint("[red3]You will only be able to use CPU rendering until you do so. [bold]This is HIGHLY unreccomended.[/bold]")
    raise SystemExit from None

#-- install libtcmalloc
setLDPL = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"
if ("LD_PRELOAD" not in os.environ) or (os.environ["LD_PRELOAD"] != setLDPL):
  print("⚙️ installing library libtcmalloc-minimal4")
  os.environ["LD_PRELOAD"] = ""
  !apt remove libtcmalloc-minimal4 &> /dev/null
  !apt install libtcmalloc-minimal4 -y &> /dev/null
  os.environ["LD_PRELOAD"] = setLDPL
else:
  print("✅ libtcmalloc-minimal4 library installed")

#-- mount drive
try:
  if os.path.exists("/content/drive"):
    print("✅ Google Drive mounted")
  else:
    print("⚙️ mounting Google Drive...")
    drive.mount("/content/drive")
except:
  print("❌ Failed to mount Google Drive...")
  raise

#-- clean empty output dirs
root = '/content/output'
if os.path.exists(root):
  folders = list(os.walk(root))[1:]
  for folder in folders:
      if not folder[2]:
          os.rmdir(folder[0])
  print("✅ Cleaned output folders")



blender_url_dict = {'2.79b'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.79/blender-2.79b-linux-glibc219-x86_64.tar.bz2",
                    '2.80rc3' : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.80/blender-2.80rc3-linux-glibc217-x86_64.tar.bz2",
                    '2.81a'   : "https://ftp.nluug.nl/pub/graphics/blender/release/Blender2.81/blender-2.81a-linux-glibc217-x86_64.tar.bz2"}

dynamic_versions = ["2.82","2.82a","2.83.0","2.83.1","2.83.2","2.83.3","2.83.4","2.83.5","2.83.6","2.83.7","2.83.8","2.83.9","2.83.10","2.83.12","2.83.13","2.83.14","2.83.15","2.83.16","2.83.17","2.83.18","2.83.19","2.83.20","2.90.0","2.90.1","2.91.0","2.91.2","2.92.0","2.93.0","2.93.1","2.93.2","2.93.3","2.93.4","2.93.5","2.93.6","2.93.7","2.93.8","2.93.9","2.93.10","2.93.11","2.93.12","2.93.13","2.93.14","2.93.15","2.93.16","2.93.17","2.93.18","3.0.0","3.0.1","3.1.0","3.1.1","3.1.2","3.2.0","3.2.1","3.2.2","3.3.0","3.3.1","3.3.2","3.3.3","3.3.4","3.3.5","3.3.6","3.3.7","3.3.8","3.4.0","3.4.1","3.5.0","3.5.1","3.6.0"]
main_ui = None



if 'config' not in globals():
  config = {
      'blender_version': '3.6.0',
      'cpu': True,
      'gpu': True,
      'optix': False,
      'animation': True,
      'anim_start': 0,
      'anim_end': 100,
      'file_source': 'gdrive',
      'gdrive_file': None,
      'upload_file': None,
      'local_file': None,
      'url_file': "",
      'output_name': "render-##",
      'zip_blend_path': "",
      'output_source': 'download',

      'gdrive_dl_selected': None,
      'gdrive_multidl_method': 'zip'
  }

import ipywidgets as widgets
from ipyfilechooser import FileChooser

def set_global(key, val):
    global config
    config[key] = val

def bind(elem, key, **kwargs):
  setter = lambda change: set_global(key, change.new)
  elem.observe(setter, names='value')
  elem.value = config[key]

  observe_value = kwargs.get('observe_value', None)
  if observe_value is not None:
    elem.observe(observe_value, names='value')
  return elem

def download_ui(out_folder, out_path, blender_out):
  path, dirs, files_folder = next(os.walk(out_folder))
  dlsource_container = widgets.VBox()
  def update_dlsource(change):
    def inlinedl(_ = None):
      if len(files_folder) == 1:
        files.download(out_folder + '/' + files_folder[0])
      else:
        print(f"⚙️ Compressing {len(files_folder)} files into .zip")
        zip_path = os.path.dirname(out_folder) + ".zip"
        if os.path.exists(zip_path): os.remove(zip_path)
        zippin = !zip -jr '{zip_path}' '{out_folder}/'
        files.download(zip_path)


    if change['new'] == 'download':
      dlb = widgets.Button(description='',button_style='',layout=widgets.Layout(width="min-content"))
      if len(files_folder) == 1:
        dlb.description = f'Download {files_folder[0]}'
      else:
        dlb.description = f'Download {len(files_folder)} files as ZIP'

      dlb.on_click(inlinedl)
      dlsource_container.children = [dlb]
    elif change['new'] == 'view':
      format = os.path.splitext(files_folder[0])[1][1:]
      if format.lower() in ['png', 'jpg', 'jpeg']:
        # print("📸 Loading preview, this might take a second...")
        file = open(out_folder + '/' + files_folder[0], "rb")
        image = file.read()
        img_widget = widgets.Image(
            value=image,
            format=format,
        )
        dlsource_container.children = [img_widget]
      else:
        dlsource_container.children = [widgets.Label(f"😔 Image preview isn't supported for .{format} files...")]
    elif change['new'] == 'gdrive':
      def save_to_gdrive(_ = None):
        if config['gdrive_dl_selected'] == None:
          print("Pick a file before saving!")
          return

        if len(files_folder) == 1:
          print("⚙️ Copying file to GDrive and flushing/remounting...")
          shutil.copy(out_folder + '/' + files_folder[0], config['gdrive_dl_selected'])
          drive.flush_and_unmount()
          drive.mount('/content/drive')
          print("✅ File saved sucessfully!")
        else:
          if config['gdrive_multidl_method'] == 'folder':
            print(f"⚙️ Copying {len(files_folder)} files to GDrive and flushing/remounting...")
            shutil.copytree(out_folder + '/', config['gdrive_dl_selected'], dirs_exist_ok=True)
            drive.flush_and_unmount()
            drive.mount('/content/drive')
            print("✅ Files saved sucessfully!")
          elif config['gdrive_multidl_method'] == 'zip':
            print(f"⚙️ Compressing {len(files_folder)} files into .zip")
            zip_path = os.path.dirname(out_folder) + ".zip"
            if os.path.exists(zip_path): os.remove(zip_path)
            zippin = !zip -jr '{zip_path}' '{out_folder}/'
            # files.download(zip_path)
            print("⚙️ Copying file to GDrive and flushing/remounting...")
            shutil.copy(zip_path, config['gdrive_dl_selected'])
            drive.flush_and_unmount()
            drive.mount('/content/drive')
            print("✅ File saved sucessfully!")

      def update_selected(change = None):
        if isinstance(change, FileChooser):
          config['gdrive_dl_selected'] = change.selected
        elif 'new_selected' in change:
          config['gdrive_dl_selected'] = change['new_selected']
        fc = FileChooser('/content/drive')
        fc.sandbox_path = '/content/drive'
        fc.register_callback(update_selected)

        savebtn = widgets.Button(description='',button_style='',layout=widgets.Layout(width="min-content"))
        savebtn.on_click(save_to_gdrive)

        if len(files_folder) == 1:
          fc.filter_pattern = ['*' + os.path.splitext(files_folder[0])[1]]
          fc.default_filename = files_folder[0]
          savebtn.description = 'Please pick a location above before saving.' if config['gdrive_dl_selected'] == None else f"Save to {config['gdrive_dl_selected']}"
          savebtn.disabled = (config['gdrive_dl_selected'] == None)
          dlsource_container.children = [fc,savebtn]
        else:
          if config['gdrive_multidl_method'] == 'zip':
            fc.show_only_dirs = False
            fc.default_filename = os.path.basename(out_folder) + ".zip"
            fc.filter_pattern = ['*.zip']
            invalid = config['gdrive_dl_selected'] == None or (not config['gdrive_dl_selected'].endswith(".zip"))
            savebtn.description = 'Please pick a location above before saving.' if invalid else f"Save {len(files_folder)} files to {config['gdrive_dl_selected']}"
            savebtn.disabled = invalid
          elif config['gdrive_multidl_method'] == 'folder':
            fc.show_only_dirs = True
            fc.filter_pattern = []
            fc.default_filename = os.path.basename(out_folder)
            invalid = config['gdrive_dl_selected'] == None or config['gdrive_dl_selected'].endswith(".zip")
            savebtn.description = 'Please pick a folder above before saving.' if invalid else f"Save {len(files_folder)} files to {config['gdrive_dl_selected']}"
            savebtn.disabled = invalid
          dlsource_container.children = [
              bind(widgets.Dropdown(
                  options=[("Save as ZIP", "zip"), ("Save files into folder", "folder")],
                  button_style='',
                  description='File-saving method',
                  style={'description_width': 'initial'}
              ), "gdrive_multidl_method", observe_value=update_selected),
              widgets.Label(f"Current {'directory' if config['gdrive_multidl_method'] == 'folder' else 'ZIP'} to save to: {config['gdrive_dl_selected']}"),
              fc, savebtn]
      update_selected({'new_selected': None})

  def print_blender_out(_ = None):
    print("\n".join(blender_out))

  print_debugbtn = widgets.Button(description='Debug - Print Blender output',button_style='',layout=widgets.Layout(width="min-content"))
  print_debugbtn.on_click(print_blender_out)

  dl_ui = widgets.VBox([
    print_debugbtn,
    widgets.HTML("<h3>Output</h3>"),
    bind(widgets.Dropdown(
      options=[('Inline Download', 'download'), ('Google Drive', 'gdrive')] + [("View image in notebook", "view")] if len(files_folder) == 1 else [],
      description="Download Method",
      style={'description_width': 'initial'}
    ), 'output_source', observe_value=update_dlsource),
    dlsource_container,
  ])
  update_dlsource({'new': config['output_source']})
  display(dl_ui)

def wget_with_progress(url, out):
  import wget
  prog = widgets.IntProgress(
    value=0.0,
    min=0.0,
    max=100.0,
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'purple'},
    orientation='horizontal'
  )
  label = widgets.Label("Downloading [?MB / ?MB]")
  view = widgets.HBox(
      [label, prog]
  )

  def bar_custom(current, total, width=80):
    prog.value = current / total * 100
    label.value ='Downloading [{0}MB / {1}MB]'.format(round(current/1000000), round(total/1000000))


  display(view)
  wget.download(url, out, bar=bar_custom)
  view.close()

def start_render(_ = None):
  main_ui.close()
  %cd -q /content
  out_folder = "output/" + datetime.now().strftime("%d-%m-%y-%H_%M_%S")
  print("📂 Setting up folders")
  os.makedirs("/content/" + out_folder)

  render_dir = '/content/render'
  if os.path.exists(render_dir) and os.path.isdir(render_dir ):
    shutil.rmtree(render_dir)
  os.makedirs(render_dir)

  file_path = ""
  if config['file_source'] == 'gdrive':
    file_path = config['gdrive_file']
  elif config['file_source'] == 'upload':
    file_path = config['upload_file']
  elif config['file_source'] == 'local':
    file_path = config['local_file']
  elif config['file_source'] == 'url':
    print("🌐 Downloading remote file to render...")
    url = config['url_file']
    !wget -nc $url
    file_path = os.path.basename(url)

  base = os.path.basename(file_path)
  if base.lower().endswith('.zip'):
    print("⚙️ Unzipping provided archive...")
    !unzip -o $uploaded_filename -d '/content/render/'
    file_path = os.path.join('/content/render', config['zip_blend_path'])
  elif base.lower().endswith('.blend'):
    shutil.copy(file_path, 'render/')
    file_path = '/content/render/' + base
  print(f"📃 Rendering {file_path}")

  ver_dynamic_url = "https://ftp.nluug.nl/pub/graphics/blender/release/Blender{0}/blender-{1}-linux-x64.tar.xz"
  ver_dynamic_url2 = "https://ftp.nluug.nl/pub/graphics/blender/release/Blender{0}/blender-{1}-linux64.tar.xz"

  ver = config['blender_version']
  blender_path = f'/content/blender_versions/{ver}'
  if not os.path.exists(blender_path):
    print(f"⚙️ Downloading Blender v{ver}")
    url = None
    if ver in blender_url_dict:
      url = blender_url_dict[ver]
    else:
      split_version = ver.split('.')
      result = '.'.join(split_version[:2])
      url = ver_dynamic_url.format(result, ver)
      res = !wget --spider --server-response $url
      faile = "Remote file does not exist" in "\n".join(res)
      if faile:
        url = ver_dynamic_url2.format(result,ver)
    # print(url)
    if os.path.exists('/content/blender_temp.tar.xz'):
      os.remove('/content/blender_temp.tar.xz')
    wget_with_progress(url, '/content/blender_temp.tar.xz')
    print("⚙️ Extracting Blender files...")
    os.makedirs(blender_path)
    !tar -xkf '/content/blender_temp.tar.xz' -C $blender_path --strip-components=1

  if os.path.exists('/content/setgpu.py'):
      os.remove('/content/setgpu.py')
  print("✏️ Writing setgpu.py script...")
  data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device, device.name)\n"+\
    "        device.use = "+str(config['gpu'] if gpu != "Unknown" else False)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(config['cpu'])+"\n"
  with open('/content/setgpu.py', 'w') as f:
    f.write(data)


  print("🚀 Starting render!")
  out_path = "/content/" + out_folder + "/" + config["output_name"]

  startf = config['anim_start']
  endf = config['anim_end']
  renderer = "OPTIX" if gpu_optix and config['optix'] else "CUDA"
  ver = config['blender_version']
  out = ""
  if config['animation']:
      if startf == endf:
          out = !'{blender_path}/blender' -b '{file_path}' -P /content/setgpu.py -E CYCLES -o '{out_path}' -noaudio -a -- --cycles-device "{renderer}"
      else:
         out =  !'{blender_path}/blender' -b '{file_path}' -P /content/setgpu.py -E CYCLES -o '{out_path}' -noaudio -s $startf -e $endf -a -- --cycles-device "{renderer}"
  else:
     out = !'{blender_path}/blender' -b '{file_path}' -P /content/setgpu.py -E CYCLES -o '{out_path}' -noaudio -f $startf -- --cycles-device "{renderer}"

  download_ui("/content/" + out_folder, out_path, out)

# Dropdown for Blender version
blender_version_dropdown = bind(widgets.Dropdown(
    options=list(blender_url_dict.keys()) + dynamic_versions,
    # value=config['blender_version'],
    description="Blender Version",
    style={'description_width': 'initial'}
), 'blender_version')

right_margin = widgets.Layout(margin='0px 8px 0px 0px')
auto_width = widgets.Layout(width="auto", margin='0px 8px 0px 0px')

render_devices = widgets.HBox([
    widgets.Label("Render Devices", layout=right_margin),
    widgets.HBox([
        bind(widgets.Checkbox(description='CPU',indent=False,layout=auto_width), 'cpu'),
        bind(widgets.Checkbox(description=f'GPU ({gpu})',indent=False,layout=auto_width, disabled=(gpu=="Unknown")), 'gpu'),
        ] + [bind(widgets.Checkbox(description='OptiX',indent=False,layout=auto_width), 'optix')] if gpu_optix else [])
], layout=widgets.Layout(align_items="center", margin='4px 0px 4px 2px'))


# Dropdown for animation config

box_lo = widgets.Layout(max_width='75px', margin='0px 8px 0px 0px')
start_frame_input = widgets.HBox([widgets.Label("Start Frame"), bind(widgets.IntText(layout=box_lo), 'anim_start')])
end_frame_input = widgets.HBox([widgets.Label("End Frame"), bind(widgets.IntText(layout=box_lo),'anim_end')])

def update_frame_inputs(change):
    if change['new'] == True:
        start_frame_input.children[1].disabled = False
        end_frame_input.children[1].disabled = False
    else:
        start_frame_input.children[1].disabled = True
        end_frame_input.children[1].disabled = True

update_frame_inputs({'new': config['animation']})
render_animation = bind(widgets.Checkbox(description='Enabled',indent=False,layout=auto_width), 'animation', observe_value=update_frame_inputs)

filesource_container = widgets.VBox()

def update_gdrive_file(new):
  if isinstance(new, FileChooser):
    config['gdrive_file'] = new.selected
    update_file_source({'new': 'gdrive'})

def update_local_file(new):
  if isinstance(new, FileChooser):
    config['local_file'] = new.selected
    update_file_source({'new': 'local'})

def upload_file(*args):
  new = files.upload()
  name = list(new.keys())
  if len(name) == 0:
    return
  else:
    name = name[0]
  if (name.endswith(".blend") or name.endswith(".zip")):
    config['upload_file'] = '/content/' + name
    update_file_source({'new': 'upload'})
  else:
    rprint("[red3]❌ That isnt a .blend or .zip file, and isn't supported...")


def update_file_source(change):
  if change['new'] == 'gdrive':
    fc = FileChooser('/content/drive')
    fc.sandbox_path = '/content/drive'
    fc.filter_pattern = ['*.blend', '*.zip']
    fc.register_callback(update_gdrive_file)

    filesource_container.children = [
            widgets.Label("Current file: " + (f"{config['gdrive_file']}" if config['gdrive_file'] is not None else 'No file selected')),
            fc
        ]
  elif change['new'] == 'local':
    fc = FileChooser('/content')
    fc.sandbox_path = '/'
    fc.show_hidden = True
    fc.filter_pattern = ['*.blend', '*.zip']
    fc.register_callback(update_local_file)

    filesource_container.children = [
            widgets.Label("Current file: " + (f"{config['local_file']}" if config['local_file'] is not None else 'No file selected')),
            fc
        ]
  elif change['new'] == 'url':
    filesource_container.children = [widgets.HBox([
        widgets.Label("URL"),
        bind(widgets.Text(
          placeholder='https://example.com/expensive_glass.blend',
          indent=False
      ),'url_file')])]
  elif change['new'] == 'upload':
    up = widgets.Button(
                description='Upload file',
                button_style='',
                tooltip='Upload file',
                icon='upload'
            )
    up.on_click(upload_file)
    filesource_container.children = [
            widgets.Label("Current file: " + (f"{config['upload_file']}" if config['upload_file'] is not None else 'No file selected')),
            up]
  update_blend_status({})


blend_container = widgets.VBox()


def update_blend_status(_ = None):
  exists = False
  error = False
  message = ""
  if config['file_source'] == 'gdrive':
    exists = os.path.exists(config['gdrive_file']) if config['gdrive_file'] is not None else False
  elif config['file_source'] == 'upload':
    exists = os.path.exists(config['upload_file']) if config['upload_file'] is not None else False
  elif config['file_source'] == 'local':
    exists = os.path.exists(config['local_file']) if config['local_file'] is not None else False

  if exists == False:
    error = True
    message = "File hasn't been selected, or the file selected doesn't exist..."

  extra_note = ""
  if gpu_optix == False:
    extra_note += "OptiX isn't supported by your runtime's GPU and so CUDA will be used."

  if gpu == "Unknown":
    if config['cpu'] == False:
      error = True
      message = "You don't have any devices to render with! Turn on the CPU, or switch to a GPU runtime..."
    else:
      extra_note += "You don't seem to be on a GPU runtime, and so you can only use the CPU."

  if config['cpu'] == False and  config['gpu'] == False:
    error = True
    message = "You don't have any devices to render with!"

  ready_msg = "No issues detected with configuration, hit the button below to render!"
  render_btn = widgets.Button(description='Render file',button_style='',tooltip='Render file')
  render_btn.on_click(start_render)
  blend_container.children = (
      [widgets.Label("Can't render yet due to issue: " + message)] if error else [
          widgets.Label(ready_msg if extra_note == "" else ready_msg + " Note: " + extra_note),
          render_btn])

update_blend_status()
update_file_source({'new': config['file_source']})


# Display the UI elements
main_ui = widgets.VBox([
    widgets.HTML("<h3>Config</h3>"),
    blender_version_dropdown,
    render_devices,
    widgets.HBox([widgets.Label("Output Name"), bind(widgets.Text(placeholder='something-##',indent=False),'output_name')]),
    widgets.HBox([widgets.Label("Blend Filepath (in ZIP)"), bind(widgets.Text(placeholder='./path/to/file.blend',indent=False),'zip_blend_path')]),
    widgets.HBox([widgets.Label("Animation", layout=right_margin), render_animation, start_frame_input, end_frame_input], layout=widgets.Layout(align_items="center", margin='0px 0px 4px 2px')),


    widgets.HTML("<h3>Blender File</h3>"),
    bind(widgets.Dropdown(
      options=[('File Upload', 'upload'), ('Google Drive', 'gdrive'), ('Remote URL', 'url'), ('Pick File on Runtime', 'local')],
      description="Source",
      style={'description_width': 'initial'}
    ), 'file_source', observe_value=update_file_source),
    filesource_container,

    widgets.HTML("<h3>Render</h3>"),
    blend_container
])
display(main_ui)
# download_ui("/content/output/28-06-23-18_16_43","/content/output/28-06-23-18_16_43/render-##", "uwu?")

CoBlend v1.0.0a by f1shy-dev

✅ ipyfilechooser package installed
✅ rich package installed
✅ ipywidgets package installed
✅ wget package installed
✅ GPU detcted - Tesla T4
✅ libtcmalloc-minimal4 library installed
✅ Google Drive mounted
✅ Cleaned output folders


In [20]:
# Reset config variable - not dangerous but will reset your options...
del config